# Example Application

This notebook is used to generate the free energy table (Table 3) in Sec. 4.4 of the manuscript. The CBFE includes a `PointMassConstraint` on `y`, whereas the BFE excludes this constraint; see code below.

In [ ]:
using ForneyLab

u_ig = [1.0, 0.0] # Ignorant policy
u_inf = [0.0, 1.0] # Informative policy
A = [0.5 1.0; 0.5 0.0] # Observation model

# Build generative model
fg = FactorGraph()

@RV y ~ Transition(placeholder(:u, dims=(2,)), placeholder(:A, dims=(2,2)))
PointMassConstraint(y) # Include for CBFE, remove for BFE
;

In [ ]:
# Generate algorithm source code
q = PosteriorFactorization(y, ids=[:Y])
algo = messagePassingAlgorithm(y, q, free_energy=true)
code = algorithmSourceCode(algo, free_energy=true)
eval(Meta.parse(code))
;

In [ ]:
# println(code) # Uncomment to inspect generated algorithm source code

In [ ]:
for u in [u_ig, u_inf] # For ignorant and informative policy respectively
    marginals = Dict{Symbol, ProbabilityDistribution}(
        :y => vague(Categorical, 2)) # Initialize marginal beliefs
    
    data = Dict(:u => u,
                :A => A) # Initialize data and parameters
    
    for i=1:10 # Perform inference
        stepY!(data, marginals)
    end
    
    F = freeEnergy(data, marginals) # Evaluate free energy
    
    println(round(F/log(2), digits=2)) # Report free energy in bits for respective policies
end